In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import seaborn as sns
import pandas as pd
import time
# Display plots inside Jupyter cell
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

# Set the dots-per-inch (resolution) of the images
mpl.rcParams['figure.dpi'] = 90

sys.path.append('/home/class_NI2021/ctxctl_contrib_0_14/')

import samna
import samna.dynapse1 as dyn1
from dynapse1constants import *
import dynapse1utils as ut
import netgen as n
import params  

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# open DYNAP-SE1 board to get Dynapse1Model
device_name = 'dynapse_nsm' #pick any name, dynapse_groupA etc...
# change the port numbers to not have conflicts with other groups

#samna.device.close_device(device_name)

model, no_gui = ut.open_dynapse1(device_name, gui=False, sender_port=18425, receiver_port=10204, select_device=True)

# silent all neurons
paramGroup = params.gen_clean_param_group()
for chip in range(4):
    for core in range(4):
        model.update_parameter_group(paramGroup, chip, core)

[0]:  Bus 3 Device 8 Dynapse1DevKit serial_number 00000027
[1]:  Bus 1 Device 79 Dynapse1DevKit serial_number 00000001
[2]:  Bus 1 Device 73 Dynapse1DevKit serial_number 00000011
[3]:  Bus 1 Device 77 Dynapse1DevKit serial_number 00000020
[4]:  Bus 1 Device 49 Dynapse1DevKit serial_number 00000000
[5]:  Bus 1 Device 50 Dynapse1DevKit serial_number 00000007
[6]:  Bus 1 Device 53 Dynapse1DevKit serial_number 00000033
Select the device you want to open by index: 4
Sender port: tcp://0.0.0.0:18425
Receiver port: tcp://0.0.0.0:10204
Opened device name: dynapse_nsm
SamnaNode ID: 1
PythonNode ID: 2


In [2]:
def spike_generator(input_type, pulse_start, pulse_stop, rate, tit=''):
    #function to create spikes
    #input_type   = 'poisson' # 'regular', 'poisson' or 'cosine' (Default: poisson)
    #pulse_start         # second - Start time of input (Default: 0)
    #pulse_stop            # second - Stop time of input (Default: 5)
    inp_duration = pulse_stop-pulse_start # second - Simulation duration (Default: 5)
    #rate             # Hz or rad/sec - Spiking rate (Default: 80 Hz for regular, 100 Hz for poission, 2 rad/sec for cosine) 
    #tit --> title for the graph; if len(tit) is 0, no plot will be displayed

    if input_type == 'regular':
    
        spikes = np.zeros(inp_duration*1000)
        dt = int(1000/rate)
        spikes[pulse_start*1000:pulse_stop*1000:dt] = 1.0

    if input_type == 'poisson':

        prob = rate * 1e-3
        mask = np.random.rand(inp_duration*1000)
    
        spikes = np.zeros(inp_duration*1000)
        spikes[mask < prob] = 1.0

    if input_type == 'cosine':
        
        spikes = np.zeros(inp_duration*1000)
        time = np.linspace(0, inp_duration, inp_duration*1000)
        co = np.cos(2 * np.pi * rate * time)
        mask = 20 * np.random.rand(inp_duration*1000)
        spikes[mask < co] = 1.0
        spikes[:pulse_start*1000]=0
        spikes[pulse_stop*1000:]=0

    if len(tit)>0:
        plt.plot(spikes,'k|')
        plt.xlabel("Time (ms)")
        plt.ylabel("Neuron ID")
        plt.ylim(0.5, 1.5)
        plt.title (tit)
        plt.show()

    spikes = np.where(spikes==1)[0]/10e2
    return spikes.tolist()

In [47]:
def param_group_DC_AMPA_GABA(chip=0, core=0):
    paramGroup = dyn1.Dynapse1ParameterGroup()

    ### General parameters ###
    # THR, gain factor of neurons
    paramGroup.param_map["IF_THR_N"].coarse_value = 5
    paramGroup.param_map["IF_THR_N"].fine_value = 80

    # refactory period of neurons
    paramGroup.param_map["IF_RFR_N"].coarse_value = 4
    paramGroup.param_map["IF_RFR_N"].fine_value = 128

    # leakage of neurons
    paramGroup.param_map["IF_TAU1_N"].coarse_value = 3
    paramGroup.param_map["IF_TAU1_N"].fine_value = 80

    # turn off tau2
    paramGroup.param_map["IF_TAU2_N"].coarse_value = 7
    paramGroup.param_map["IF_TAU2_N"].fine_value = 255

    # turn off DC
    paramGroup.param_map["IF_DC_P"].coarse_value = 0
    paramGroup.param_map["IF_DC_P"].fine_value = 0
    
    ### Chip and core-spefific parameters ###
#     if chip == 0 and core == 0:
#         # leakage of AMPA
#         paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 2
#         paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 10

#         # gain of AMPA
#         paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
#         paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

#         # weight of AMPA
#         paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 6
#         paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
    
    # states
    if chip == 1 and core == 1:
        # leakage of AMPA
        paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
        paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 20

        # gain of AMPA
        paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
        paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

        # weight of AMPA
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 6
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
        
        # leakage of GABA_B
        paramGroup.param_map["NPDPII_TAU_S_P"].coarse_value = 2
        paramGroup.param_map["NPDPII_TAU_S_P"].fine_value = 80

        # gain of GABA_B
        paramGroup.param_map["NPDPII_THR_S_P"].coarse_value = 6
        paramGroup.param_map["NPDPII_THR_S_P"].fine_value = 100

        # weight of GABA_B
        paramGroup.param_map["PS_WEIGHT_INH_S_N"].coarse_value = 7
        paramGroup.param_map["PS_WEIGHT_INH_S_N"].fine_value = 200
        
    # inh1
    if chip == 1 and core == 2:
        # leakage of AMPA
        paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
        paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 40

        # gain of AMPA
        paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
        paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

        # weight of AMPA
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
        
    # inh2    
    if chip == 1 and core == 3:
        # leakage of AMPA
        paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
        paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 40

        # gain of AMPA
        paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
        paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

        # weight of AMPA
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 6
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
        
        # leakage of GABA_B
        paramGroup.param_map["NPDPII_TAU_S_P"].coarse_value = 2
        paramGroup.param_map["NPDPII_TAU_S_P"].fine_value = 80

        # gain of GABA_B
        paramGroup.param_map["NPDPII_THR_S_P"].coarse_value = 6
        paramGroup.param_map["NPDPII_THR_S_P"].fine_value = 100

        # weight of GABA_B
        paramGroup.param_map["PS_WEIGHT_INH_S_N"].coarse_value = 7
        paramGroup.param_map["PS_WEIGHT_INH_S_N"].fine_value = 200
    
    # transitions
    if chip == 2 and core == 1:
        # leakage of AMPA
        paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
        paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 50

        # gain of AMPA
        paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
        paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

        # weight of AMPA
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
        
        # leakage of GABA_B
        paramGroup.param_map["NPDPII_TAU_S_P"].coarse_value = 2
        paramGroup.param_map["NPDPII_TAU_S_P"].fine_value = 80

        # gain of GABA_B
        paramGroup.param_map["NPDPII_THR_S_P"].coarse_value = 6
        paramGroup.param_map["NPDPII_THR_S_P"].fine_value = 100

        # weight of GABA_B
        paramGroup.param_map["PS_WEIGHT_INH_S_N"].coarse_value = 6
        paramGroup.param_map["PS_WEIGHT_INH_S_N"].fine_value = 200
    
    # s_sum
    if chip == 2 and core == 2:
        # leakage of AMPA
        paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
        paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 60

        # gain of AMPA
        paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
        paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

        # weight of AMPA
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
    
    # in_wta
    if chip == 2 and core == 3:
        # leakage of AMPA
        paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
        paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 70

        # gain of AMPA
        paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
        paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

        # weight of AMPA
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
        
    if chip == 0 and core == 0:   
        # leakage of AMPA
        paramGroup.param_map["NPDPIE_TAU_F_P"].coarse_value = 4
        paramGroup.param_map["NPDPIE_TAU_F_P"].fine_value = 70

        # gain of AMPA
        paramGroup.param_map["NPDPIE_THR_F_P"].coarse_value = 5
        paramGroup.param_map["NPDPIE_THR_F_P"].fine_value = 150

        # weight of AMPA
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 7
        paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 200
#     # weight of AMPA
#     paramGroup.param_map["PS_WEIGHT_EXC_F_N"].coarse_value = 0
#     paramGroup.param_map["PS_WEIGHT_EXC_F_N"].fine_value = 0
        
    return paramGroup

<img src="network-base.png" width="45%"/>

In [53]:
paramGroup = params.gen_clean_param_group()
for chip in range(4):
    for core in range(4):
        model.update_parameter_group(paramGroup, chip, core)

net_gen = n.NetworkGenerator()

net_gen.clear_network()

# inputs (excitatory)
spikegen_ids = [(0, 0, 1)]
spikegens = []
for spikegen_id in spikegen_ids:
    spikegens.append(n.Neuron(spikegen_id[0], True))

#Tims testing shizzle
test_n = n.Neuron(0,0,12)    
poissongen = n.Neuron(0,0,18,True)
net_gen.add_connection(spikegens[0], test_n, dyn1.Dynapse1SynType.AMPA)
net_gen.add_connection(poissongen, test_n, dyn1.Dynapse1SynType.AMPA)
net_gen.add_connection(poissongen, transitions[0], dyn1.Dynapse1SynType.AMPA)
spikegen_ids = [(0,0,18)]
global_poisson_gen_ids = ut.get_global_id_list(spikegen_ids)
poisson_gen = model.get_poisson_gen()
poisson_gen.set_chip_id(0)
poisson_gen.write_poisson_rate_hz(global_poisson_gen_ids[0], 200)
param_group11 = param_group_DC_AMPA_GABA(0, 0)

states = [n.Neuron(1, 1, 11), n.Neuron(1, 1, 12), n.Neuron(1, 1, 13)] # s_1, s_2, s_3
neurons1 = [n.Neuron(1, 2, 21), n.Neuron(1, 2, 22), n.Neuron(1, 2, 23)] # first group of inh neurons
neurons2 = [n.Neuron(1, 3, 31), n.Neuron(1, 3, 32), n.Neuron(1, 3, 33)] # second group of inh neurons
transitions = [n.Neuron(2, 1, 41), n.Neuron(2, 1, 42), n.Neuron(2, 1, 44)] # t_1, t_2, t_3
s_sum = n.Neuron(2, 2, 51)
in_wta = n.Neuron(2, 3, 61)

# Connections
#Spikegen activating transitions
net_gen.add_connection(spikegens[0], transitions[0], dyn1.Dynapse1SynType.AMPA) # input -> t_1
net_gen.add_connection(spikegens[0], transitions[1], dyn1.Dynapse1SynType.AMPA) # input -> t_2
net_gen.add_connection(spikegens[0], transitions[2], dyn1.Dynapse1SynType.AMPA) # input -> t_3


"""
#Transition states
net_gen.add_connection(transitions[0], states[1], dyn1.Dynapse1SynType.AMPA) # t_1 -> s_2
net_gen.add_connection(transitions[1], states[2], dyn1.Dynapse1SynType.AMPA) # t_2 -> s_3
net_gen.add_connection(transitions[2], states[0], dyn1.Dynapse1SynType.AMPA) # t_3 -> s_1

#Recursive state amp
net_gen.add_connection(states[0], states[0], dyn1.Dynapse1SynType.AMPA) # s_1 -> s_1
net_gen.add_connection(states[1], states[1], dyn1.Dynapse1SynType.AMPA) # s_2 -> s_2
net_gen.add_connection(states[2], states[2], dyn1.Dynapse1SynType.AMPA) # s_3 -> s_3

#states to sum
net_gen.add_connection(states[0], s_sum, dyn1.Dynapse1SynType.AMPA) # s_1 -> s_sum
net_gen.add_connection(states[1], s_sum, dyn1.Dynapse1SynType.AMPA) # s_2 -> s_sum
net_gen.add_connection(states[2], s_sum, dyn1.Dynapse1SynType.AMPA) # s_3 -> s_sum

#Sum to inhibitory WTA & to inihbitory AND 
net_gen.add_connection(s_sum, in_wta, dyn1.Dynapse1SynType.AMPA) # s_sum -> in_wta
net_gen.add_connection(s_sum, neurons2[0], dyn1.Dynapse1SynType.AMPA) # s_sum -> inh21
net_gen.add_connection(s_sum, neurons2[1], dyn1.Dynapse1SynType.AMPA) # s_sum -> inh22
net_gen.add_connection(s_sum, neurons2[2], dyn1.Dynapse1SynType.AMPA) # s_sum -> inh23

#WTA inhibiting states
net_gen.add_connection(in_wta, states[0], dyn1.Dynapse1SynType.GABA_B) # in_wta -> s_1
net_gen.add_connection(in_wta, states[1], dyn1.Dynapse1SynType.GABA_B) # in_wta -> s_2
net_gen.add_connection(in_wta, states[2], dyn1.Dynapse1SynType.GABA_B) # in_wta -> s_3

#States activating inhib 1
net_gen.add_connection(states[0], neurons1[0], dyn1.Dynapse1SynType.AMPA) # s_1 -> inh11
net_gen.add_connection(states[1], neurons1[1], dyn1.Dynapse1SynType.AMPA) # s_2 -> inh12
net_gen.add_connection(states[2], neurons1[2], dyn1.Dynapse1SynType.AMPA) # s_3 -> inh13

#I2 inhibiting I2
net_gen.add_connection(neurons1[0], neurons2[0], dyn1.Dynapse1SynType.GABA_B) # inh11 -> inh21
net_gen.add_connection(neurons1[1], neurons2[1], dyn1.Dynapse1SynType.GABA_B) # inh11 -> inh22
net_gen.add_connection(neurons1[2], neurons2[2], dyn1.Dynapse1SynType.GABA_B) # inh11 -> inh23

#I2 inhibiting transition
net_gen.add_connection(neurons2[0], transitions[0], dyn1.Dynapse1SynType.GABA_B) # inh21 -> t_1
net_gen.add_connection(neurons2[1], transitions[1], dyn1.Dynapse1SynType.GABA_B) # inh22 -> t_2
net_gen.add_connection(neurons2[2], transitions[2], dyn1.Dynapse1SynType.GABA_B) # inh23 -> t_3
"""
new_config = net_gen.make_dynapse1_configuration()
model.apply_configuration(new_config)

param_groupTEST = param_group_DC_AMPA_GABA(0, 0)

param_group11 = param_group_DC_AMPA_GABA(1, 1)
model.update_parameter_group(param_group11, 1, 1)
param_group12 = param_group_DC_AMPA_GABA(1, 2)
model.update_parameter_group(param_group12, 1, 2)
param_group13 = param_group_DC_AMPA_GABA(1, 3)
model.update_parameter_group(param_group13, 1, 3)
param_group21 = param_group_DC_AMPA_GABA(2, 1)
model.update_parameter_group(param_group21, 2, 1)
param_group22 = param_group_DC_AMPA_GABA(2, 2)
model.update_parameter_group(param_group22, 2, 2)
param_group23 = param_group_DC_AMPA_GABA(2, 3)
model.update_parameter_group(param_group23, 2, 3)

# config3 = model.get_configuration()
# for chip_idx in range(2):
#     for core_idx in range(3):
#         print('chip' + str(chip_idx+1) +', core' + str(core_idx+1) + ' ' + str(config3.chips[chip_idx+1].cores[core_idx+1].parameter_group.param_map['IF_DC_P'].fine_value))
time.sleep(1)

In [54]:
#-----------------------------------spike generator-------------------------------------

fpga_spike_gen = model.get_fpga_spike_gen()
spike_times_input = spike_generator('poisson', 0, 5, 100)
spike_times = spike_times_input.copy()

device_indices = len(spike_times) * [spikegens[0].neuron_id]

# Spike times need to be sorted for FPGA
spike_times, device_indices= zip(*sorted(zip(spike_times, device_indices)))

# The chips where the post neurons are
target_chips = [0] * len(spike_times_input)

isi_base = 900
repeat_mode = False
ut.set_fpga_spike_gen(fpga_spike_gen, spike_times, device_indices, target_chips, isi_base, repeat_mode)

In [55]:
monitored_neurons = [(1, 1, 11), (1, 1, 12), (1, 1, 13), # s_1, s_2, s_3
                     (1, 2, 21), (1, 2, 22), (1, 2, 23), # first group of inh neurons
                     (1, 3, 31), (1, 2, 32), (1, 2, 33), # second group of inh neurons
                     (2, 1, 41), (2, 1, 42), (2, 1, 44), # t_1, t_2, t_3
                     (2, 2, 51), # s_sum
                     (2, 3, 61), (0,0,12)] # in_wta

graph, filter_node, sink_node = ut.create_neuron_select_graph(model, monitored_neurons)

graph.start()
sink_node.get_events()
fpga_spike_gen.start() # Start spike generator
poisson_gen.start()
duration = 5 # This is the duration of the recording
time.sleep(duration)
events = sink_node.get_events()
print(len(events),"events.")
graph.stop()
fpga_spike_gen.stop()  # Stop spike generator
poisson_gen.stop()
evts_n = np.array([[evt.timestamp, evt.neuron_id] for evt in events])
timestamp_offset = 0 
df_spikes = pd.DataFrame()
for evt in events:
    if timestamp_offset==0:
        timestamp_offset = evt.timestamp
    spikeData = dict(chip = evt.chip_id, 
                     core = evt.core_id,
                     neuron_id = evt.neuron_id,
                     times =(evt.timestamp-timestamp_offset)* 1e-6)
    df_spikes = pd.concat([df_spikes,pd.DataFrame(spikeData,index=[0])])

0 events.


In [56]:
df_spikes

""


In [57]:
neuron_ids = list(set(df_spikes['neuron_id'].values))
num_subplots = len(neuron_ids)

fig, axs = plt.subplots(num_subplots, 1, figsize=(16, 8))
plt.subplots_adjust(wspace=None, hspace=1)

for i in range(len(neuron_ids)):
    neuron_id = neuron_ids[i]
    df1 = df_spikes[df_spikes['neuron_id'] == neuron_id]
    chip = df1.chip.values[0]
    core = df1.core.values[0]
    
    if num_subplots > 1:
        axs[i].plot(df1['times'].values, df1['neuron_id'].values, 'k|')
        axs[i].set_xlabel("time (s)")
        axs[i].set_ylabel("neuron id")
        axs[i].set_title('chip ' + str(chip) + ', core ' + str(core) + ', neuron_id ' + str(neuron_id))
    else:
        axs.plot(df1['times'].values, df1['neuron_id'].values, 'k|')
        axs.set_xlabel("time (s)")
        axs.set_ylabel("neuron id")
        axs.set_title('chip ' + str(chip) + ', core ' + str(core) + ', neuron_id ' + str(neuron_id))

KeyError: 'neuron_id'

In [119]:
# def plot_all(df, chip, core):
#     fig, axs = plt.subplots(3, 1, figsize=(16, 8))
#     plt.subplots_adjust(wspace=None, hspace=1)

#     df1 = df[(df['chip'] == chip) and (df['core'] == core)]
#     neuron_ids = list(set(df1['neuron_id'].values))
    
#     for i in range(len(neuron_ids)):
#         neuron_id = neuron_ids[i]
        
#         chip = df1.chip.values[0]
#         core = df1.core.values[0]

#         axs[i].plot(df1['times'].values, df1['neuron_id'].values, 'k|')
#         axs[i].set_xlabel("time (s)")
#         axs[i].set_ylabel("neuron id")
#         axs[i].set_title('chip ' + str(chip) + ', core ' + str(core) + ', neuron_id ' + str(neuron_id))